In [1]:
import tensorflow as tf
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.33)  
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)) 
# keras 或 tensorflow 调用GPU报错：Blas GEMM launch failed
# https://blog.csdn.net/Leo_Xu06/article/details/82023330
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allocator_type = 'BFC' #A "Best-fit with coalescing" algorithm, simplified from a version of dlmalloc.
config.gpu_options.per_process_gpu_memory_fraction = 0.3
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config)) 

Using TensorFlow backend.


In [2]:
# Convolutional Neural Network

# Installing Theano
# pip install --upgrade --no-deps git+git://github.com/Theano/Theano.git

# Installing Tensorflow
# pip install tensorflow

# Installing Keras
# pip install --upgrade keras

# Part 1 - Building the CNN

# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

# Initialising the CNN
classifier = Sequential()

# Step 1 - Convolution
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))

# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Adding a second convolutional layer
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Step 3 - Flattening
classifier.add(Flatten())

# Step 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [3]:
# Part 2 - Fitting the CNN to the images

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

classifier.fit_generator(training_set,
                         steps_per_epoch = 8000,
                         epochs = 25,
                         validation_data = test_set,
                         validation_steps = 2000)

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Epoch 1/25
8000/8000 [==============================] - 868s 109ms/step - loss: 0.3566 - acc: 0.8336 - val_loss: 0.6140 - val_acc: 0.7856
Epoch 2/25
8000/8000 [==============================] - 832s 104ms/step - loss: 0.1078 - acc: 0.9591 - val_loss: 0.9303 - val_acc: 0.8063
Epoch 3/25
8000/8000 [==============================] - 865s 108ms/step - loss: 0.0539 - acc: 0.9810 - val_loss: 1.1860 - val_acc: 0.8001
Epoch 4/25
8000/8000 [==============================] - 849s 106ms/step - loss: 0.0385 - acc: 0.9868 - val_loss: 1.1963 - val_acc: 0.8122oss: 0.0385 - acc: 0.98
Epoch 5/25
8000/8000 [==============================] - 857s 107ms/step - loss: 0.0314 - acc: 0.9895 - val_loss: 1.2988 - val_acc: 0.8010s: 0. -  - ETA: 3s - loss: 0.0314 - - ETA: 2s - 
Epoch 6/25
8000/8000 [==============================] - 857s 107ms/step - loss: 0.0258 - acc: 0.9913 - val_loss: 1.3859 - val_acc: 0.8000 loss: 0.0259 - ac

In [4]:
# Part 3 - Making new predictions

import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
    prediction = 'dog'
else:
    prediction = 'cat'